In [7]:
import pandas as pd
import numpy as np
from glob import glob
import shutil


In [3]:
predict_maskrcnn = 'Submission/submission_31.1_1.csv'
predict_2_class = 'Submission/2-cls test pred.csv'

In [4]:
submission_maskrcnn = pd.read_csv(predict_maskrcnn)
submission_2_class= pd.read_csv(predict_2_class)

In [5]:
submission_maskrcnn.head()

,image_id,PredictionString
0,002a34c58c5b758217ed1f584ccbcfe9,3 0.993 810 1286 1900 1584 11 0.948 622 307 84...
1,004f33259ee4aef671c2b95d54e4be68,3 0.998 988 1478 1853 1774 0 0.994 1292 609 15...
2,008bdde2af2462e86fd373a445d0f4cd,3 0.999 1125 1513 1908 1732 0 0.999 1462 821 1...
3,009bc039326338823ca3aa84381f17f1,3 0.999 665 1115 1550 1295 10 0.97 218 1181 69...
4,00a2145de1886cb9eb88869c85d74080,3 1.0 813 1349 1773 1575 0 0.993 1118 710 1343...


In [6]:
submission_2_class.head()

,image_id,target
0,002a34c58c5b758217ed1f584ccbcfe9,0.013326
1,004f33259ee4aef671c2b95d54e4be68,0.037235
2,008bdde2af2462e86fd373a445d0f4cd,0.939700
3,009bc039326338823ca3aa84381f17f1,0.123799
4,00a2145de1886cb9eb88869c85d74080,0.654006


## Filter threshold

In [48]:
low_thr  = 0.05
high_thr = 0.8

In [49]:
pred = pd.merge(submission_maskrcnn, submission_2_class, on = 'image_id', how = 'left')
pred

,image_id,PredictionString,target
0,002a34c58c5b758217ed1f584ccbcfe9,3 0.993 810 1286 1900 1584 11 0.948 622 307 84...,0.013326
1,004f33259ee4aef671c2b95d54e4be68,3 0.998 988 1478 1853 1774 0 0.994 1292 609 15...,0.037235
2,008bdde2af2462e86fd373a445d0f4cd,3 0.999 1125 1513 1908 1732 0 0.999 1462 821 1...,0.939700
3,009bc039326338823ca3aa84381f17f1,3 0.999 665 1115 1550 1295 10 0.97 218 1181 69...,0.123799
4,00a2145de1886cb9eb88869c85d74080,3 1.0 813 1349 1773 1575 0 0.993 1118 710 1343...,0.654006
...,...,...,...
2995,ff91fb82429a27521bbec8569b041f02,3 1.0 1242 1440 2382 1722 0 0.999 1626 660 187...,0.936325
2996,ff9fcc4087ed5e941209aa3fa948e364,0 0.995 1107 708 1413 1035 3 0.92 774 1406 165...,0.963583
2997,ffaa288c8abca300974f043b57d81521,11 0.984 1535 352 2117 589 0 0.92 1247 1479 18...,0.178720
2998,ffc441e0c8b7153844047483a577e7c3,3 0.999 721 1318 1533 1591 11 0.89 1059 295 12...,0.225196


In [50]:
pred['PredictionString'].value_counts().iloc[[0]]

14 1.0 0 0 1 1    66
Name: PredictionString, dtype: int64

In [51]:
def filter_2cls(row, low_thr=low_thr, high_thr=high_thr):
    prob = row['target']
    if prob<low_thr:
        ## Less chance of having any disease
        row['PredictionString'] = '14 1 0 0 1 1'
    elif low_thr<=prob<high_thr:
        ## More change of having any diesease
        row['PredictionString']+=f' 14 {prob} 0 0 1 1'
    elif high_thr<=prob:
        ## Good chance of having any disease so believe in object detection model
        row['PredictionString'] = row['PredictionString']
    else:
        raise ValueError('Prediction must be from [0-1]')
    return row

In [52]:
sub = pred.apply(filter_2cls, axis=1)
sub.head()

,image_id,PredictionString,target
0,002a34c58c5b758217ed1f584ccbcfe9,14 1 0 0 1 1,0.013326
1,004f33259ee4aef671c2b95d54e4be68,14 1 0 0 1 1,0.037235
2,008bdde2af2462e86fd373a445d0f4cd,3 0.999 1125 1513 1908 1732 0 0.999 1462 821 1...,0.939700
3,009bc039326338823ca3aa84381f17f1,3 0.999 665 1115 1550 1295 10 0.97 218 1181 69...,0.123799
4,00a2145de1886cb9eb88869c85d74080,3 1.0 813 1349 1773 1575 0 0.993 1118 710 1343...,0.654006


In [53]:
sub['PredictionString'].value_counts().iloc[[0]]

14 1 0 0 1 1    1700
Name: PredictionString, dtype: int64

In [54]:
import os
os.getcwd()

'/home/daitran/Desktop/git/chest_x_ray_abnormalities_detection/MaskRCNN_implementation'

In [55]:
sub[['image_id', 'PredictionString']].to_csv('submission_threshold_2.csv',index = False)

In [27]:
df_down_filtered = pd.read_csv('Submission/submission_31.1_filtered.csv')

In [28]:
df_down_filtered

,image_id,PredictionString
0,002a34c58c5b758217ed1f584ccbcfe9,14 1 0 0 1 1
1,004f33259ee4aef671c2b95d54e4be68,14 1 0 0 1 1
2,008bdde2af2462e86fd373a445d0f4cd,3 0.999 1125 1513 1908 1732 0 0.999 1462 821 1...
3,009bc039326338823ca3aa84381f17f1,3 0.999 665 1115 1550 1295 10 0.97 218 1181 69...
4,00a2145de1886cb9eb88869c85d74080,3 1.0 813 1349 1773 1575 0 0.993 1118 710 1343...
...,...,...
2995,ff91fb82429a27521bbec8569b041f02,3 1.0 1242 1440 2382 1722 0 0.999 1626 660 187...
2996,ff9fcc4087ed5e941209aa3fa948e364,0 0.995 1107 708 1413 1035 3 0.92 774 1406 165...
2997,ffaa288c8abca300974f043b57d81521,11 0.984 1535 352 2117 589 0 0.92 1247 1479 18...
2998,ffc441e0c8b7153844047483a577e7c3,3 0.999 721 1318 1533 1591 11 0.89 1059 295 12...


In [29]:
df_down_filtered['PredictionString'].value_counts().iloc[[0]]

14 1 0 0 1 1    1912
Name: PredictionString, dtype: int64